<a href="https://colab.research.google.com/github/Alirezamirbagheri/Machine-Learning-Transfer-learning-for-configurations-control/blob/main/Transfer_Learning_for_Configuration_Control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Libraries**

In [6]:
import numpy as np
import PIL.Image as Image
import matplotlib.pylab as plt
import torch
from torch.utils.data import Dataset
import pathlib
from torchsummary import summary
from tqdm import tqdm
from torch.optim import Adam
import torch.nn as nn
from torchvision.transforms import transforms
import os
!pip install torchmetrics
from torchmetrics import ConfusionMatrix

In [7]:
print("Numpy version: " + np.__version__)
print("PIL.Image version: " + Image.__version__)
print("Matplotlib.pylab version: " + plt.__version__)

Numpy version: 2.0.2
PIL.Image version: 11.3.0
Matplotlib.pylab version: 2.0.2


# **2. Data preparation and import**
In this Notebook Google Drive is used.

In [8]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## 2.1 **Loading the test data**
The data set is already split into a **training**, **validation** and  **test** sets.  The class names are derived from the sub folder names.

For a CNN, the images must have the same dimensions (height × width × channels).
Because:
- Convolution layers slide fixed-size filters over the image.
- If images had different sizes, the output feature maps would also have different shapes → impossible to batch them together.
- Dense (fully connected) layers at the end require a fixed input size.

Here we define a variable called **IMAGE_SHAPE**.

In [9]:
import tensorflow as tf
IMAGE_SHAPE = (224, 224)
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
training_data="./drive/MyDrive/ML/Transfer_Learning/dataset_truck/training"
training_image_data  = image_generator.flow_from_directory(training_data,target_size=IMAGE_SHAPE)

# ImageDataGenerator, a utility for preprocessing and augmenting image data before feeding it into a neural network.
# Automatically load images from disk. Preprocess them (e.g., rescale, rotate, flip, shift, etc.). Feed batches of preprocessed images into your model during training.
# This makes a generator but the data not loaded yet
# This normalization helps neural networks train more efficiently and stably.
# rescale=1./255 → normalizes pixel values from [0,255] → [0,1].
# target_size=(224,224) → resizes every image to the input size required by your CNN (e.g., ResNet, MobileNet, VGG all expect 224×224).
# flow_from_directory → automatically assigns labels from folder names inside training_data.
# Feed batches of preprocessed images into your model during training.

Found 60 images belonging to 6 classes.


In [10]:
# Hyperparameters
BATCH = 32
EPOCH = 5

## 2.2 Loading the validation data

In [11]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
validation_data="./drive/MyDrive/ML/Transfer_Learning/dataset_truck/validation"
validation_data_image_data  = image_generator.flow_from_directory(validation_data,target_size=IMAGE_SHAPE)

Found 150 images belonging to 6 classes.


# **3. The pre-trained base model**
A pre-trained model is used for Transfer Learning.
For the use of this model there are two possibilities:


* Feature Extractor: Use of the learned features for your own application.
* Fine Tuning: Re-training of the Base Model for your own application.

In the following, we will use the base model as a feature extractor and create our own classification layer for our application.

## 4.1 Structure and use of the basic model - MobilNetV2
Various pre-trained network architectures are already available for download. Often these are pre-trained with the [ImageNet dataset](http://www.image-net.org/).

Here we use the MobileNetV2 model as a pre-trained base model. The torchsummary.summary() command can be used to output the architecture of the model to get an overview. It is necessary to give a network and a shape of images as inputs.

If we want to use a model as a feature extractor, we need to fit it to our own classification task. The classification layer of the pre-trained model must thus be replaced by one of our own.

To simplify this, we load a version of MobilNetV2, which is already prepared for use as a feature extractor - the classification layer is already removed.